In [8]:
import torch 
from torch.utils import data
from torch import nn

from typing import List, Tuple, Dict, Union, Optional
from typing import Iterable

import pandas as pd
import numpy as np
import surprise

In [2]:
from load_data import get_validation_data
from utils import Dataset

In [3]:
validation_data, heristic = get_validation_data()
validation_data

,user_0,user_1,user_2,user_id,item_0,item_1,item_2,item_id,rating
0,-0.189219,0.688799,0.496749,0,-7.189302,-17.358819,-8.191424,0,3.0
1,-0.189219,0.688799,0.496749,0,0.749574,0.957448,2.106965,1,9.0
2,-0.189219,0.688799,0.496749,0,-3.443982,-0.042470,-1.148656,2,6.0
3,-0.189219,0.688799,0.496749,0,-1.101991,-0.047565,-0.453863,3,6.0
4,-0.189219,0.688799,0.496749,0,7.837388,18.125961,19.356996,4,9.0
...,...,...,...,...,...,...,...,...,...
24995,-0.401447,-0.463426,-1.824159,249,-1.780516,-3.164613,0.289502,95,7.0
24996,-0.401447,-0.463426,-1.824159,249,-1.835749,-1.362072,-0.549780,96,8.0
24997,-0.401447,-0.463426,-1.824159,249,-0.900141,-0.433615,0.459035,97,6.0
24998,-0.401447,-0.463426,-1.824159,249,-1.414088,2.804230,-37.497444,98,10.0


In [30]:
data_sparse = validation_data #validation_data.sample(frac=0.05)
data_sparse = data_sparse.sample(frac=0.9)
data_train = data_sparse.sample(frac=0.8)
data_test = data_sparse.drop(data_train.index)

heu_train = heristic.loc[data_train.index]
heu_test = heristic.loc[data_test.index]

In [31]:
dataset_train = Dataset(data_train, 
                  validation_data.user_id.unique().shape[0], 
                  validation_data.item_id.unique().shape[0], 
                  user_based=True,
                  long_matrix=True,
                  user_column="user_id", item_column="item_id", rating_column="rating")

dataset_test = Dataset(data_test, 
                  validation_data.user_id.unique().shape[0], 
                  validation_data.item_id.unique().shape[0], 
                  user_based=True,
                  long_matrix=True,
                  user_column="user_id", item_column="item_id", rating_column="rating")

heuset_train = Dataset(heu_train, 
                  validation_data.user_id.unique().shape[0], 
                  validation_data.item_id.unique().shape[0], 
                  user_based=True,
                  long_matrix=True,
                  user_column="user_id", item_column="item_id", rating_column="rating")

heuset_test = Dataset(heu_test, 
                  validation_data.user_id.unique().shape[0], 
                  validation_data.item_id.unique().shape[0], 
                  user_based=True,
                  long_matrix=True,
                  user_column="user_id", item_column="item_id", rating_column="rating")

In [32]:
from models import CollobarativeModel

In [33]:
svd = CollobarativeModel(surprise.SVD, "SVD")
svd.train(dataset_train)
svd.test(dataset_test)

RMSE: 1.0505
MAE:  0.5351


{'rmse': 1.0505247635043073,
 'mae': 0.5350650813442086,
 'ndcg': 0.8399529364935231}

In [34]:
nmf = CollobarativeModel(surprise.NMF, "NMF")
nmf.train(dataset_train)
nmf.test(dataset_test)

RMSE: 1.4846
MAE:  1.0781


{'rmse': 1.4845595319303118,
 'mae': 1.07809049874095,
 'ndcg': 0.8297430632507045}

In [35]:
knn = CollobarativeModel(surprise.KNNBasic, "kNN")
knn.train(dataset_train)
knn.test(dataset_test)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.1662
MAE:  0.6648


{'rmse': 1.1662037317613203,
 'mae': 0.6647512120117873,
 'ndcg': 0.8342587704586688}